In [1]:
import sys
sys.path.append('../../../')
sys.path.append('/home/sheida.rahnamai/GIT/HDN/')
import torch
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tifffile import imread
import glob
import matplotlib.patches as patches
import hdbscan as hd
from hdbscan import prediction
from sklearn.cluster import HDBSCAN
from collections import Counter
from openTSNE import TSNE
from openTSNE import TSNEEmbedding
from openTSNE import affinity
from openTSNE import initialization
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
# %reload lib.plotting 
from lib import plotting as p
from lib import dataprep as dp
import random
import umap
from tqdm import tqdm
from lib.evaluation import FeatureExtractor
# from lib.dataloader import CustomTestDataset
from lib.dataloader import CustomDataset
import tifffile as tiff
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
import os
from boilerplate import boilerplate
from importlib import reload
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix, adjusted_rand_score, normalized_mutual_info_score, homogeneity_score, completeness_score, v_measure_score

In [2]:
data_dir = "/group/jug/Sheida/pancreatic beta cells/download/"
keys = ['high_c1', 'high_c2', 'high_c3']

# train data
train_img_paths = [os.path.join(data_dir, "train/", img, f"{img}_source.tif") for img in keys]
train_lbl_paths = [os.path.join(data_dir, "train/", img, f"{img}_gt.tif") for img in keys]
train_images = {img: tiff.imread(path) for img, path in zip(keys, train_img_paths)}
train_labels = {img: tiff.imread(path) for img, path in zip(keys, train_lbl_paths)}
#val data
val_img_paths = [os.path.join(data_dir, "val/", img, f"{img}_source.tif") for img in keys]
val_lbl_paths = [os.path.join(data_dir, "val/", img, f"{img}_gt.tif") for img in keys]
val_images = {img: tiff.imread(path) for img, path in zip(keys, val_img_paths)}
val_labels = {img: tiff.imread(path) for img, path in zip(keys, val_lbl_paths)}


for key in tqdm(keys, desc='Filtering data'):
   train_images[key], train_labels[key] = boilerplate._filter_slices(train_images[key], train_labels[key])
   val_images[key], val_labels[key] = boilerplate._filter_slices(val_images[key], val_labels[key])

Filtering data: 100%|██████████| 3/3 [00:00<00:00,  4.73it/s]


5x5 <br>
Uncategorized: 60827 Nucleus: 18054 Granule: 17870 Mitochondria: 7438 <br>
Uncategorized: 6980 Nucleus: 1853 Granule: 2109 Mitochondria: 882

5x5 - 3x3 OR 3x3 <br>
Uncategorized: 72213 Nucleus: 18462 Granule: 24558 Mitochondria: 9355 <br>
Uncategorized: 8314 Nucleus: 1901 Granule: 2885 Mitochondria: 1090

5x5 - 1x1 OR 3x3 - 1x1 OR 1x1<br>
Uncategorized: 84693 Nucleus: 18816 Granule: 32293 Mitochondria: 11368 <br>
Uncategorized: 9748 Nucleus: 1943 Granule: 3839 Mitochondria: 1313

In [3]:
def save_percent(file_path, data):
    with open(file_path, 'wb') as file:
        # Serialize and write the variable to the file
        pickle.dump(data, file)

In [4]:
import random

patch_size = 64
modes = ['5x5', '3x3', '1x1']
categories = ['uncategorized', 'nucleus', 'granule', 'mitochondria']

for i in range(len(modes)):
    mask_size = int(modes[i][0])
    label_size = int(modes[i][-1])
    print(f"Mask size: {mask_size} Label size: {label_size}")
    
    train_set = CustomDataset(train_images, train_labels, patch_size, mask_size, label_size)
    val_set = CustomDataset(val_images, val_labels, patch_size, mask_size, label_size)
    print('Datasets created!!!')
    
    t_uncategorize = train_set.patches_by_label[0]
    t_nucleus = train_set.patches_by_label[1]
    t_granule = train_set.patches_by_label[2]
    t_mitochondria = train_set.patches_by_label[3]
    print(f'Train size for {categories[0]}: {len(t_uncategorize)}, {categories[1]}: {len(t_nucleus)}, {categories[2]}: {len(t_granule)}, {categories[3]}: {len(t_mitochondria)}')
    
    v_uncategorize = val_set.patches_by_label[0]
    v_nucleus = val_set.patches_by_label[1]
    v_granule = val_set.patches_by_label[2]
    v_mitochondria = val_set.patches_by_label[3]
    print(f'Val size for {categories[0]}: {len(v_uncategorize)}, {categories[1]}: {len(v_nucleus)}, {categories[2]}: {len(v_granule)}, {categories[3]}: {len(v_mitochondria)}')
    
    t0 = random.sample(t_uncategorize, len(t_uncategorize)//10)
    save_percent(data_dir+f'train/{modes[i]}/10_percent_{categories[0]}.pickle',t0)
    save_percent(data_dir+f'train/{modes[i]}/1_percent_{categories[0]}.pickle',random.sample(t0, len(t0)//10))
    
    t1 = random.sample(t_nucleus, len(t_nucleus)//10)
    save_percent(data_dir+f'train/{modes[i]}/10_percent_{categories[1]}.pickle',t1)
    save_percent(data_dir+f'train/{modes[i]}/1_percent_{categories[1]}.pickle',random.sample(t1, len(t1)//10))
    
    t2 = random.sample(t_granule, len(t_granule)//10)
    save_percent(data_dir+f'train/{modes[i]}/10_percent_{categories[2]}.pickle',t2)
    save_percent(data_dir+f'train/{modes[i]}/1_percent_{categories[2]}.pickle',random.sample(t2, len(t2)//10))
    
    t3 = random.sample(t_mitochondria, len(t_mitochondria)//10)
    save_percent(data_dir+f'train/{modes[i]}/10_percent_{categories[3]}.pickle',t3)
    save_percent(data_dir+f'train/{modes[i]}/1_percent_{categories[3]}.pickle',random.sample(t3, len(t3)//10))
    
    print(f'10% of the train set for {categories[0]}: {len(t0)}, {categories[1]}: {len(t1)}, {categories[2]}: {len(t2)}, {categories[3]}: {len(t3)}')
    print(f'1% of the train set for {categories[0]}: {len(t0)//10}, {categories[1]}: {len(t1)//10}, {categories[2]}: {len(t2)//10}, {categories[3]}: {len(t3)//10}')
    
    v0 = random.sample(v_uncategorize, len(v_uncategorize)//10)
    save_percent(data_dir+f'val/{modes[i]}/10_percent_{categories[0]}.pickle',v0)
    save_percent(data_dir+f'val/{modes[i]}/1_percent_{categories[0]}.pickle',random.sample(v0, len(v0)//10))
    
    v1 = random.sample(v_nucleus, len(v_nucleus)//10)
    save_percent(data_dir+f'val/{modes[i]}/10_percent_{categories[1]}.pickle',v1)
    save_percent(data_dir+f'val/{modes[i]}/1_percent_{categories[1]}.pickle',random.sample(v1, len(v1)//10))
    
    v2 = random.sample(v_granule, len(v_granule)//10)
    save_percent(data_dir+f'val/{modes[i]}/10_percent_{categories[2]}.pickle',v2)
    save_percent(data_dir+f'val/{modes[i]}/1_percent_{categories[2]}.pickle',random.sample(v2, len(v2)//10))
    
    v3 = random.sample(v_mitochondria, len(v_mitochondria)//10)
    save_percent(data_dir+f'val/{modes[i]}/10_percent_{categories[3]}.pickle',v3)
    save_percent(data_dir+f'val/{modes[i]}/1_percent_{categories[3]}.pickle',random.sample(v3, len(v3)//10))
    
    print(f'10% of the val set for {categories[0]}: {len(v0)}, {categories[1]}: {len(v1)}, {categories[2]}: {len(v2)}, {categories[3]}: {len(v3)}')
    print(f'1% of the val set for {categories[0]}: {len(v0)//10}, {categories[1]}: {len(v1)//10}, {categories[2]}: {len(v2)//10}, {categories[3]}: {len(v3)//10}')

Mask size: 5 Label size: 5


Extracting patches from high_c1: 38it [00:00, 379.87it/s]

Extracting patches from high_c1: 965it [00:02, 370.23it/s]
Extracting patches from high_c2: 889it [00:02, 399.69it/s]
Extracting patches from high_c3: 907it [00:02, 384.74it/s]
Extracting patches from high_c1: 109it [00:00, 431.93it/s]
Extracting patches from high_c2: 101it [00:00, 455.24it/s]
Extracting patches from high_c3: 103it [00:00, 414.26it/s]


Datasets created!!!
Train size for uncategorized: 60827, nucleus: 18054, granule: 17870, mitochondria: 7438
Val size for uncategorized: 6980, nucleus: 1853, granule: 2109, mitochondria: 882
10% of the train set for uncategorized: 6082, nucleus: 1805, granule: 1787, mitochondria: 743
1% of the train set for uncategorized: 608, nucleus: 180, granule: 178, mitochondria: 74
10% of the val set for uncategorized: 698, nucleus: 185, granule: 210, mitochondria: 88
1% of the val set for uncategorized: 69, nucleus: 18, granule: 21, mitochondria: 8
Mask size: 5 Label size: 3


Extracting patches from high_c1: 965it [00:02, 335.37it/s]
Extracting patches from high_c2: 889it [00:02, 337.02it/s]
Extracting patches from high_c3: 907it [00:02, 381.89it/s]
Extracting patches from high_c1: 109it [00:00, 434.97it/s]
Extracting patches from high_c2: 101it [00:00, 456.70it/s]
Extracting patches from high_c3: 103it [00:00, 158.83it/s]


...

Datasets created!!!
Train size for uncategorized: 72213, nucleus: 18462, granule: 24558, mitochondria: 9355
Val size for uncategorized: 8314, nucleus: 1901, granule: 2885, mitochondria: 1090
10% of the train set for uncategorized: 7221, nucleus: 1846, granule: 2455, mitochondria: 935
1% of the train set for uncategorized: 722, nucleus: 184, granule: 245, mitochondria: 93
10% of the val set for uncategorized: 831, nucleus: 190, granule: 288, mitochondria: 109
1% of the val set for uncategorized: 83, nucleus: 19, granule: 28, mitochondria: 10
Mask size: 3 Label size: 1


Extracting patches from high_c1: 965it [00:02, 410.42it/s]
Extracting patches from high_c2: 889it [00:02, 331.58it/s]
Extracting patches from high_c3: 907it [00:03, 297.04it/s]
Extracting patches from high_c1: 109it [00:00, 395.07it/s]
Extracting patches from high_c2: 101it [00:00, 420.84it/s]
Extracting patches from high_c3: 103it [00:00, 380.49it/s]


Datasets created!!!
Train size for uncategorized: 84693, nucleus: 18816, granule: 32293, mitochondria: 11368
Val size for uncategorized: 9748, nucleus: 1943, granule: 3839, mitochondria: 1313
10% of the train set for uncategorized: 8469, nucleus: 1881, granule: 3229, mitochondria: 1136
1% of the train set for uncategorized: 846, nucleus: 188, granule: 322, mitochondria: 113
10% of the val set for uncategorized: 974, nucleus: 194, granule: 383, mitochondria: 131
1% of the val set for uncategorized: 97, nucleus: 19, granule: 38, mitochondria: 13
Mask size: 1 Label size: 1


Extracting patches from high_c1: 965it [00:03, 315.16it/s]
Extracting patches from high_c2: 889it [00:02, 437.83it/s]
Extracting patches from high_c3: 907it [00:03, 273.32it/s]
Extracting patches from high_c1: 109it [00:00, 339.08it/s]
Extracting patches from high_c2: 101it [00:00, 428.94it/s]
Extracting patches from high_c3: 103it [00:00, 380.78it/s]


Datasets created!!!
Train size for uncategorized: 84693, nucleus: 18816, granule: 32293, mitochondria: 11368
Val size for uncategorized: 9748, nucleus: 1943, granule: 3839, mitochondria: 1313
10% of the train set for uncategorized: 8469, nucleus: 1881, granule: 3229, mitochondria: 1136
1% of the train set for uncategorized: 846, nucleus: 188, granule: 322, mitochondria: 113
10% of the val set for uncategorized: 974, nucleus: 194, granule: 383, mitochondria: 131
1% of the val set for uncategorized: 97, nucleus: 19, granule: 38, mitochondria: 13
